In [1]:
from numpy.random import randint
from numpy.random import rand
from numpy.random import normal
from random import uniform

In [2]:
def objective(x):
    '''
    rosenbrock function
    
    :param 
     x : coordinates(x,y)
     
    :returns
     value of rosenbrock function at X(x,y)
    '''
    a = 1
    b = 15
    return ((a - x[0]**2)+b*((x[1]-x[0]**2)**2))

In [3]:
def crossover(p1, p2, r_cross):
    '''
    Crossover function
    
    :params
     p1 : parent 1
     p2 : parent 2
     r_cross : rate of crossover
     
    :returns
     c1 : child 1
     c2 : child 2
    '''
    c1, c2 = p1.copy(), p2.copy()
    r = uniform(0, 1.1)
    while(r > 1):
        r = uniform(0, 1.1)
    if r < r_cross:
        c1[0],c1[1] = p1[0], p2[1]
        c2[0],c2[1] = p2[0], p1[1]
    return c1,c2

In [4]:
def mutation(c, r_mut):
    '''
    mutates the child
    
    :params
     c    : offspring to mutate
     r_mut: rate of mutation
     
    :returns
     p : offspring
     truth value if mutation was successful or not
    '''
    p = c.copy()
    r = uniform(0, 1.1)
    while(r > 1):
        r = uniform(0, 1.1)
    if r < r_mut:
        idx = randint(0,2)
        p[idx] = float(normal(5,1,1))
        return p, True
    return p, False

In [5]:
def tournament(pop):
    '''
    The crossover function requires two parents to be selected from the population pool. 
    The Tournament is used to do this. Two individuals are selected from the population 
    pool and a random number in [0, 1] is chosen. If this number is less than the 
    'selection rate' (e.g. 0.85), then the fitter individual is selected; otherwise, the 
    weaker one is selected.
    
    :params
     pop : population
    
    :returns
     individual who won the tournament
    '''
    c1 = pop[randint(0,len(pop)-1)]
    c2 = pop[randint(0,len(pop)-1)]
    f1 = objective(c1)
    f2 = objective(c2)
    
    if f1>f2:
        fittest = c1
        weakest = c2
    else:
        fittest = c2
        weakest = c1
        
    selection_rate = 0.85
    r = uniform(0,1.1)
    while r>1:
        r = uniform(0,1.1)
    if r < selection_rate:
        return fittest
    else:
        return weakest

In [9]:
def genetic_algorithm(objective, bounds, n_iter, n_pop, r_cross, r_mut):
    '''
    GA to optimize objective function
    
    :params
     objective : objective function
     n_iter    : number of generation/number of iterations
     n_pop     : size of population
     r_cross   : rate of crossover
     r_mut     : rate of mutation
     
    :returns
     best      : best individual
     best_eval : score of best individual
    '''
    
    pop = []
    
    # initialize population
    for i in range(n_pop):
        individual = []
        for i in range(2):
            num = float(normal(10,2,1))
            while num < bounds[i][0] and num > bounds[i][1]:
                num = float(normal(10,2,1))
            individual.append(num)
        pop.append(individual)
    

    Nm = 0                                     # number of mutations           
    phi = 0
    sigma = 1
    best = pop[0]
    best_eval = objective(pop[0])
    
    for gen in range(n_iter):
        scores = [objective(p) for p in pop]   # get scores of all individuals in the population
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
        print("\n>Generation %d, new best f(%s) = %f" % (gen, best, best_eval))
        
        next_pop = []
        for i in range(0,n_pop,2):
            # conduct tournament to get 2 parents for crossover
            p1 = tournament(pop)             
            p2 = tournament(pop)
            
            # perform crossover
            c1,c2 = crossover(p1,p2,1.0)
            
            # mutate child 1
            old_val = objective(c1)
            c1,success = mutation(c1,r_mut)
            if success:
                Nm += 1
                if objective(c1) < old_val:
                    phi = phi + 1
            
            # mutate child 2
            old_val = objective(c2)
            c2,success = mutation(c2,r_mut)
            if success:
                Nm += 1
                if objective(c2) < old_val:
                    phi = phi + 1
            
            # add children to next generation population
            next_pop.append(c1)
            next_pop.append(c2)
        
        print('Total Number of mutations:', Nm)
        if(Nm == 0):
            phi = 0
        else:
            phi = phi / Nm
        if(phi < 0.2):
            sigma = sigma/0.998
        elif(phi > 0.2):
            sigma = sigma*0.998
        
        # Calculate new adaptive mutation rate to stop too much mutation..
        r_mut = abs(normal(loc=0.0, scale=sigma, size=None))
        while r_mut > 0.2:
            r_mut = abs(normal(loc=0.0, scale=sigma, size=None))
        print('New r_mut:',r_mut)
        Nm = 0
        phi = 0
        
        # checks if population is stale
        if next_pop == pop:
            print('Stale')
            
        pop = next_pop
    return best, best_eval

In [10]:
bounds = [(-20,20),(-20,20)]
# total iterations or generations
n_iter = 100
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 0.06

# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_iter, n_pop, r_cross, r_mut)
print('Done!')

print('f(%s) = %f' % (best, score))


>Generation 0, new best f([5.965200402594902, 8.398873948056824]) = 11050.569263
Total Number of mutations: 5
New r_mut: 0.18631361491797904

>Generation 1, new best f([4.203630407260952, 11.67589385471392]) = 522.360581
Total Number of mutations: 12
New r_mut: 0.10309126680081959

>Generation 2, new best f([3.7581114286086574, 10.188089245996549]) = 219.176838
Total Number of mutations: 13
New r_mut: 0.043723836767971536

>Generation 3, new best f([3.466455316582394, 10.037822983383014]) = 47.699997
Total Number of mutations: 3
New r_mut: 0.10684580270532879

>Generation 4, new best f([3.466455316582394, 10.037822983383014]) = 47.699997
Total Number of mutations: 14
New r_mut: 0.039286967440486

>Generation 5, new best f([3.306131431744111, 10.9613513578875]) = -9.916233
Total Number of mutations: 2
New r_mut: 0.03223922177647788

>Generation 6, new best f([3.306131431744111, 10.9613513578875]) = -9.916233
Total Number of mutations: 2
New r_mut: 0.18257642392113071

>Generation 7, ne